In [ ]:
# does not work anymore because removed mongo integration, need to query from pinecone

In [3]:
import os
import pinecone
from langchain.vectorstores import Pinecone
from Database import CollectionWrapperPine
from tqdm import tqdm 
import datetime


# embedding model parameters
BATCH_SIZE = 100

index = CollectionWrapperPine()._index

for batch in tqdm(range(0, len(docs_list)//100 +1)):
    batch_list = docs_list[batch*BATCH_SIZE:min(len(docs_list), (batch+1)*BATCH_SIZE)]
    vectors = []
    for doc in batch_list:
        document = doc.copy()
        if "desc_embedding" in document:
            document.pop("desc_embedding")
        lat = doc['location'][0]
        long = doc['location'][1]
        document.pop("location")
        document['latitude'] = lat
        document['longitude'] = long
        
        document["date_added"] = datetime.datetime.strftime(document["date_added"], "%Y-%m-%d")
        
        id = document["_id"]
        document.pop("_id")
        
        vectors.append(
            {
            'id': str(id),
            'values': np.zeros(DIM).tolist(),
            'metadata': document
            })
    if len(vectors) > 0:
        upsert_response = index.upsert(vectors)
    if upsert_response.failed:
        print(upsert_response.errors)



100%|██████████| 118/118 [01:11<00:00,  1.64it/s]


In [7]:
import json

entries = {}
for doc in docs_list:
    document = doc.copy()
    if "desc_embedding" in document:
        document.pop("desc_embedding")
    lat = doc["location"][0]
    long = doc["location"][1]
    document.pop("location")
    document["latitude"] = lat
    document["longitude"] = long

    document["date_added"] = datetime.datetime.strftime(
        document["date_added"], "%Y-%m-%d"
    )

    id = document["_id"]
    document.pop("_id")

    entries.update(
        {
            str(id): {
                "id": str(id),
                "values": np.zeros(DIM).tolist(),
                "metadata": document,
            }
        }
    )
with open("pinecone-entries.json", "w") as f:
    json.dump(entries, f, indent=4)